In [1]:
import sys
sys.path.append('/home/g/Desktop/git/nodules/src/')
sys.path.append('/home/g/Desktop/git/nodules/tf/')
sys.path.append('/home/g/Desktop/git/nodules/keras/')
sys.path.append('/home/g/Desktop/git/keras/keras/preprocessing/')

In [2]:
import init
import xsmallModel as xm
import numpy as np

Using TensorFlow backend.


In [3]:
cfg = ['/scr/nodules/no4/train/','/scr/nodules/no4/train/']
trainImages, trainLabels = init.loadData(cfg)

LOADING DATA...


In [4]:
import kerasUtil as ku

In [5]:
print len(trainImages)
print len(trainLabels)

trainImages, trainLabels = ku.cleanData(trainImages, trainLabels)

print len(trainImages)
print len(trainLabels)

968
968
966
966


In [6]:
trainImages = ku.to3d(trainImages)

In [7]:
trainLabels = ku.get3Labels(trainLabels)
trainImages = ku.convertArray(trainImages)

In [8]:
cfg = ['/scr/nodules/no4/test/','/scr/nodules/no4/test/']
testImages, testLabels = init.loadData(cfg)
testImages, testLabels = ku.cleanData(testImages, testLabels)
testImages = ku.to3d(testImages)
testLabels = ku.get3Labels(testLabels)
testImages = ku.convertArray(testImages)

LOADING DATA...


In [9]:
print testImages.shape
print testLabels.shape
print trainImages.shape
print trainLabels.shape

(214, 1, 16, 64, 64)
(214, 3)
(966, 1, 16, 64, 64)
(966, 3)


In [10]:
from keras import backend as K
from keras.models import Sequential 
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution3D, MaxPooling3D
from keras.layers.advanced_activations import LeakyReLU
from image3D import ImageDataGenerator
from keras.models import load_model

In [11]:
def getModel():
  model = Sequential()
  model.add(Convolution3D(128,kernel_size=5,strides=(1,2,2),data_format='channels_first',input_shape=(1,16,64,64),kernel_initializer='glorot_normal',use_bias=True))
  model.add(LeakyReLU(alpha=0.01))
  model.add(MaxPooling3D(pool_size=(2,2,2)))
  model.add(Convolution3D(192,kernel_size=4,strides=1,data_format='channels_first',kernel_initializer='glorot_normal',use_bias=True))
  model.add(LeakyReLU(alpha=0.01))
  model.add(MaxPooling3D(pool_size=(2,2,2)))
  model.add(Dropout(0.5))
  model.add(Flatten())
  model.add(Dense(96))
  model.add(LeakyReLU(alpha=0.01))
  model.add(Dropout(0.5))
  model.add(Dense(48))
  model.add(LeakyReLU(alpha=0.01))
  model.add(Dropout(0.5))
  model.add(Dense(3))
  model.add(Activation('softmax'))
  return model


In [12]:
print (np.sum(trainLabels[:,0])/trainLabels.shape[0])
print (np.sum(trainLabels[:,1])/trainLabels.shape[0])
print (np.sum(trainLabels[:,2])/trainLabels.shape[0])

print (np.sum(testLabels[:,0])/testLabels.shape[0])
print (np.sum(testLabels[:,1])/testLabels.shape[0])
print (np.sum(testLabels[:,2])/testLabels.shape[0])

0.364389233954
0.635610766046
0.0
0.345794392523
0.654205607477
0.0


In [13]:
datagen = ImageDataGenerator(
    featurewise_center=True, #center image mean for dataset at zero
    featurewise_std_normalization=True, #divide image by dataset std dev
    zca_whitening=False, #TODO: TRY PCA WHITENING
    rotation_range=0, #TODO: AT LEAST ONE MODEL WITH DATA FROM ALL POSSIBLE AXES & ROTATION
    width_shift_range=(4./trainImages.shape[4]), #TODO: LOOK AT IMPLEMENTATION TO VERIFY
    height_shift_range=(4./trainImages.shape[4]), #TODO: VERIFY SHIFT IS EQUAL TO CROPPING
    fill_mode='constant', #fill region outside boundaries w/ constant
    cval=0, #set at image mean zero
    shear_range=0., #set to zero for no effect
    zoom_range=0., #set to zero for no effect
    data_format='channels_first')

datagen.fit(trainImages)

model = getModel()
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [14]:
loss = 1.0
while loss > 0.05:
  model.fit_generator(datagen.flow(trainImages,trainLabels,batch_size=32),
                      steps_per_epoch=trainImages.shape[0]/32,epochs=10)
  loss, acc = model.evaluate(testImages,testLabels,batch_size=32)
  ct += 10
  print "\nEPOCH: " + str(ct) + "  LOSS: " + str(loss) + "  ACCURACY: " + str(acc)
  model.save('bestModel042417-epoch'+str(ct)+'.h5')   

Epoch 1/10
30/30 [==============================] - 6s - loss: 1.0624 - acc: 0.5708      

NameError: name 'ct' is not defined